## Final Project Submission

Please fill out:
* Student name: Isabella Siele
* Student pace:  full time
* Scheduled project review date/time: 3/10/2025
* Instructor name: Samuel Karu
* Blog post URL:


# Aviation Accident Risk Analysis
## 1.Overview

The company is planning to expand into the aviation industry to diversify its portfolio. The idea is to purchase and operate aircraft for commercial and private use. The project will thus use data to provide insights and guide on decisions to make by cleaning, analysing and visualizations.

## 2.Problem Statement

The company does not have much information on aviation safety or accident patterns. This may expose the business to unnecessary risks like financial losses and safety concerns.Thus the  project aims to determine which aircraft has the lowest risk for the company to start this new business and what factors to consider which influence the occurence of accidents. 

## 3.Objectives

This project will focus on answering three main questions:

-Which aircraft make and categories have the lowest accident and fatality rates?It helps to identify safe aircraft types to prioritize during purchase.

-What outside factors(Weather conditions)play the biggest role  in accidents?

-How does the purpose of flight(commercial)affect accident risks?



## Data Understanding

In [ ]:
# Importing needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# loading data
df = pd.read_csv(".\data\Aviation_Data.csv")

In [ ]:
# Top 5 rows in the dataset
df.head()

In [ ]:
# last 5 rows in the dataset
df.tail()

In [ ]:
# Shows Number of rows and columns in dataset
df.shape

In [ ]:
# Info of dataset
df.info()

In [ ]:
# Columns in dataset
df.columns

In [ ]:
# The datatypes in columns
df.dtypes

## Data Preparation

In [ ]:
# checking for missing values
df.isna().sum().sort_values(ascending=False)

In [ ]:
# Dropping the first five because alot of data is missing
df.drop(columns=["Schedule","Air.carrier","FAR.Description","Longitude","Latitude"], inplace=True)

In [ ]:
# Select columns which will be used
use_columns = [
    "Make","Event.Date",
    "Model",
    "Injury.Severity",
    "Total.Fatal.Injuries",
    "Total.Serious.Injuries",
    "Total.Minor.Injuries",
    "Total.Uninjured",
    "Purpose.of.flight",
    "Weather.Condition","Broad.phase.of.flight"
]

In [ ]:
#nMissing values in the selected columns
missing_count = df[use_columns].isnull().sum()
missing_percentage = df[use_columns].isnull().mean()

# Combine to a dataframe
missing = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing Percentage": missing_percentage
})
missing

In [ ]:
# Dropping missing values in Make, Model,Purpose of flight,Weather condition,broad phase of flight
df.dropna(subset=["Make","Model","Purpose.of.flight","Weather.Condition","Broad.phase.of.flight"], inplace=True)

In [ ]:
# Fill injury columns blanks with 0
injury_columns = [
    "Total.Fatal.Injuries", "Total.Serious.Injuries", 
            "Total.Minor.Injuries", "Total.Uninjured"
]
df[injury_columns] = df[injury_columns].fillna(0)

In [ ]:
# Changing injury columns datatype to integer
df[injury_columns] = df[injury_columns].astype(int)

In [ ]:
# Recheck missing values 
df[use_columns].isna().sum().sort_values(ascending=False) 

In [ ]:
# Changing the categorical columns datatype to string 
df[["Make","Model","Purpose.of.flight","Weather.Condition","Broad.phase.of.flight"]] = df[[ "Make","Model","Purpose.of.flight","Weather.Condition","Broad.phase.of.flight"]].astype(str)

In [ ]:
# Convert date columns to datetime
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')
df['Publication.Date'] = pd.to_datetime(df['Publication.Date'], errors='coerce')
df["Event.Date"].dtypes

In [ ]:
# Checking for duplicates
df.duplicated().value_counts()

In [ ]:
# Dropping duplicated values
df.drop_duplicates(inplace=True)

In [ ]:
# Ensuring duplicates were dropped
df.duplicated().value_counts()

In [ ]:
# Making all to be uppercase and stripping spaces
df["Purpose.of.flight"] = df["Purpose.of.flight"].astype(str).str.upper().str.strip()

In [ ]:
# Regrouping the purpose of flight
df["Purpose.of.flight"] = df["Purpose.of.flight"].replace({
    "BUSINESS": "COMMERCIAL","EXECUTIVE/CORPORATE":"COMMERCIAL",
    "PUBS":"COMMERCIAL","PUBL":"COMMERCIAL",
    "AIR RACE/SHOW":"AIR SHOW","AIR RACE SHOW":"AIR SHOW",
    "PUBLIC AIRCRAFT _ LOCAL":"PUBLIC AIRCRAFT",
    "PUBLIC AIRCRAFT - STATE":"PUBLIC AIRCRAFT",
    "PUBLIC AIRCRAFT - LOCAL":"PUBLIC AIRCRAFT",
    "PUBLIC AIRCRAFT - FEDERAL":"PUBLIC AIRCRAFT"})

In [ ]:
# Regrouping manufacturers of aircrafts
df["Make"] = df["Make"].replace({"BOMBARDIER":"BOMBARDIER INC"})

In [ ]:
# Getting year using the event date
df["Year"] = df["Event.Date"].dt.year

In [ ]:
# Calculating fatality rate and creating a new column Fatality rate
# Fatality rate = [Total Fatal Injuries]/ [Total Aboard]
df['Total.Aboard'] = (
    df['Total.Fatal.Injuries'] +df['Total.Serious.Injuries'] +
    df['Total.Minor.Injuries'] +df['Total.Uninjured'])

df['Fatality.Rate'] = df['Total.Fatal.Injuries'] / df['Total.Aboard']

In [ ]:
# Top 5 rows to check if changes were implemented
df.head()

## Data Analysis

- Distribution of accidents over time

In [ ]:
# Accidents by year
accidents_per_year = df["Year"].value_counts().sort_index()

plt.figure(figsize=(15,6))
accidents_per_year.plot(kind="line", marker="o")
plt.title("Number of Accidents Per Year")
plt.xlabel("Year")
plt.ylabel("Accident Count")
plt.grid(True)
plt.show()

This shows that the number of accidents has been decreasing over the years but the beginning is flat due to no records of accidents early on.

- Distribution of accidents by phase of flight 

In [ ]:
# Accidents by Flight Phase
phase_counts = df["Broad.phase.of.flight"].value_counts()

plt.figure(figsize=(8,6))
phase_counts.plot(kind="bar")
plt.title("Accidents by Broad Phase of Flight")
plt.ylabel("Accident Count")
plt.xlabel("Phase")
plt.show()

Most accidents are seen to occur during the landing phase followed by takeoff then cruise. It means they should increase safety precautions during landing and takeoff.

- Comparison of Accident Counts and Fatalities Across Aircraft Manufacturers

In [ ]:
# Top 10 makes with most accidents
top_makes = df['Make'].value_counts().nlargest(10).index
accidents = df['Make'].value_counts().loc[top_makes]

# Average fatalities for the same top makes
fatalities = df.groupby("Make")["Total.Fatal.Injuries"].mean().loc[top_makes]

fig, ax1 = plt.subplots(figsize=(12,6))

# Bars (accident counts)
ax1.bar(top_makes, accidents, color='skyblue')
ax1.set_xlabel("Aircraft Make")
ax1.set_ylabel("Number of Accidents")

# Second y-axis (fatalities)
ax2 = ax1.twinx()
ax2.plot(top_makes, fatalities, color='red', marker='o',)
ax2.set_ylabel("Average Fatalities per Accident", color='red')

plt.title("Accidents vs Fatality Rate by Aircraft Make")
plt.xticks(rotation=45)
plt.show()


This shows that Cessna has a lot of accidents  but the average fatality per accident is very relatively low .The fatality rate per accident is moderate compared to boeing which has a very low number of accidents but the fatality is very high.

- Accidents and fatalities by purpose of flight

In [ ]:
# Group by Purpose of Flight
purpose_stats = df.groupby("Purpose.of.flight").agg(
    accidents=("Event.Id", "count"),
    avg_fatality_rate=("Fatality.Rate", "mean"))

# Filter out categories with very few accidents
purpose_stats = purpose_stats[purpose_stats["accidents"] >= 10]


In [ ]:
# Sort by accident count in descending order
purpose_stats = purpose_stats.sort_values("accidents", ascending=False)


fig, ax1 = plt.subplots(figsize=(12,6))

bars = ax1.bar(purpose_stats.index, purpose_stats["accidents"],)
ax1.set_ylabel("Accident Count")
ax1.set_xlabel("Purpose of Flight")
ax1.tick_params(rotation=45) 

ax2 = ax1.twinx()
ax2.plot(purpose_stats.index, purpose_stats["avg_fatality_rate"], 
         color="red", marker="o", label="Fatality Rate")
ax2.set_ylabel("Average Fatality Rate", color="red", fontsize=12)

plt.title("Accident Count vs Fatality Rate by Purpose of Flight", )
plt.show()

Personal and instructional flights are seen to have a high number of accidents but a very low fatility rate compared to commercial flights which are seen to have few accidents but a very high fatality rate. This may be due to the large number of passengers in commercial flights compared to private flights.

- Analysis of accidents based on weather condition

In [ ]:
# Group data by weather condition
weather_stats = df.groupby("Weather.Condition").agg(
    accidents=("Event.Id", "count"),
    fatalities=("Total.Fatal.Injuries", "sum"),
    uninjured=("Total.Uninjured", "sum"),
    avg_fatality_rate=("Fatality.Rate", "mean"))

# Sort by accident counts
weather_stats = weather_stats.sort_values("accidents", ascending=False)

fig, ax1 = plt.subplots(figsize=(10,6))
# Bar plot for accident counts
bars = ax1.bar(weather_stats.index, weather_stats["accidents"])
ax1.set_ylabel("Accident Count")
ax1.set_xlabel("Weather Condition")

# Line plot for fatality rate
ax2 = ax1.twinx()
ax2.plot(weather_stats.index, weather_stats["avg_fatality_rate"], color="red", marker="o")
ax2.set_ylabel("Fatality Rate")

plt.title("Accident Count vs Fatality Rate by Weather Condition")
plt.show()


It shows that most accidents occured during VMC which is when most flights are conducted but it has a relatively small fatality rate.It is then shown that most fatalities occured during IMC though it has fewer accidents